In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings

In [2]:
accre_unfiltered = pd.read_csv('accre-gpu-jobs-2022-v2.csv', on_bad_lines='skip')
accre_unfiltered

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [3]:
maxwell_unfiltered =  accre_unfiltered[accre_unfiltered['PARTITION'] == 'maxwell']
maxwell_unfiltered_df = pd.DataFrame(maxwell_unfiltered)
maxwell_unfiltered = accre_unfiltered[accre_unfiltered['PARTITION'] == 'maxwell']

In [4]:
maxwell_unfiltered['STATE'].value_counts().head(10)

COMPLETED              127043
CANCELLED by 353986       304
CANCELLED by 855431       300
RUNNING                   171
FAILED                    117
CANCELLED                  85
CANCELLED by 885837        37
CANCELLED by 756088        37
CANCELLED by 885613        24
CANCELLED by 196703        13
Name: STATE, dtype: int64

Maxwell has the best success rate by a wide margin. 127k completions to 117 fails.

In [5]:
pascal_unfiltered =  accre_unfiltered[accre_unfiltered['PARTITION'] == 'pascal']
pascal_unfiltered_df = pd.DataFrame(maxwell_unfiltered)
pascal_unfiltered = accre_unfiltered[accre_unfiltered['PARTITION'] == 'pascal']

In [6]:
pascal_unfiltered['STATE'].value_counts().head(10)

COMPLETED              81314
CANCELLED by 9201      41637
RUNNING                 1021
CANCELLED by 353986      564
CANCELLED                256
CANCELLED by 891307      234
FAILED                   230
CANCELLED by 887337       51
CANCELLED by 885613       40
CANCELLED by 879523       32
Name: STATE, dtype: int64

Pascal has the worst by far. 81k completions to 230 fails.

In [7]:
turing_unfiltered =  accre_unfiltered[accre_unfiltered['PARTITION'] == 'turing']
turing_unfiltered_df = pd.DataFrame(maxwell_unfiltered)
turing_unfiltered = accre_unfiltered[accre_unfiltered['PARTITION'] == 'turing']

In [8]:
turing_unfiltered['STATE'].value_counts().head(10)

COMPLETED              74976
CANCELLED by 174423     6116
CANCELLED                606
CANCELLED by 891307      274
CANCELLED by 887337      157
CANCELLED by 756088      148
FAILED                    93
CANCELLED by 885837       83
CANCELLED by 885613       48
RUNNING                   45
Name: STATE, dtype: int64

Turing is in the middle. 75k completions to 93 fails.

In [9]:
accre = pd.DataFrame(accre_unfiltered)
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [10]:
accre['USEDMEM'] = accre['USEDMEM'].str.replace(r'[a-zA-Z]','')
accre

C:\Users\garre\AppData\Local\Temp\ipykernel_6220\1215905320.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  accre['USEDMEM'] = accre['USEDMEM'].str.replace(r'[a-zA-Z]','')


,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [11]:
accre['REQTIME'] = accre['REQTIME'].astype(str)
accre['USEDTIME'] = accre['USEDTIME'].astype(str)
accre['REQTIME'] = pd.to_timedelta(accre['REQTIME'].str.replace('-',' days '))
accre['USEDTIME'] = pd.to_timedelta(accre['USEDTIME'].str.replace('-',' days '))
accre['REQTIME'] = pd.to_timedelta(accre['REQTIME']).dt.total_seconds()
accre['USEDTIME'] = pd.to_timedelta(accre['USEDTIME']).dt.total_seconds()
accre = accre[accre['USEDMEM'].map(len) > 1]
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,172800.0,793.0,1,2,1,pascal,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,18000.0,537.0,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,72000.0,18586.0,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,54000.0,15070.0,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40,302400.0,67776.0,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,432000.0,52.0,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70,432000.0,2091.0,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21,432000.0,243.0,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52,432000.0,673.0,1,1,1,turing,0:0,COMPLETED


In [12]:
maxwell =  accre[accre['PARTITION'] == 'maxwell']
maxwell_df = pd.DataFrame(maxwell)

In [13]:
maxwell = accre[accre['PARTITION'] == 'maxwell']
maxwell

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
2,32880718,glasshouse,brady,402.32,18000.0,537.0,1,3,1,maxwell,0:0,COMPLETED
244,32883667,glasshouse,miya,374.38,14400.0,966.0,1,1,1,maxwell,127:0,FAILED
462,32885245,glasshouse,brady,404.90,18000.0,537.0,1,3,1,maxwell,0:0,COMPLETED
467,32885270,glasshouse,brady,403.18,18000.0,528.0,1,3,1,maxwell,0:0,COMPLETED
508,32885377,glasshouse,brodie,188.82,43200.0,165.0,1,1,1,maxwell,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
335127,46542252,glasshouse,miya,6.57,172800.0,34.0,1,6,4,maxwell,0:0,COMPLETED
335856,46577601,glasshouse,tamara,1.98,288000.0,8.0,2,12,6,maxwell,0:0,COMPLETED
335916,46582283,glasshouse,jonnie,2068.21,172800.0,172838.0,1,6,4,maxwell,0:9,CANCELLED
335984,46586533,glasshouse,tamara,1.79,288000.0,5.0,2,12,6,maxwell,0:0,COMPLETED


In [14]:
maxwell['USEDMEM'].value_counts()

5.53       24156
5.54        9188
7.42        2426
7.41         359
7.53         133
           ...  
1135.12        1
1070.57        1
1180.81        1
1118.98        1
1.77           1
Name: USEDMEM, Length: 36985, dtype: int64

In [15]:
maxwell['STATE'].value_counts()

COMPLETED              125371
FAILED                     80
CANCELLED                  44
CANCELLED by 885613         4
CANCELLED by 196703         1
CANCELLED by 859439         1
Name: STATE, dtype: int64

In [16]:
maxwell.describe()

,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,1.255010e+05,125501.000000,125501.000000,125501.000000,125501.000000
mean,5.433242e+04,1862.469693,1.002908,1.135704,1.047275
std,8.108995e+04,19812.537301,0.085430,1.250943,0.592964
min,6.000000e+01,0.000000,1.000000,1.000000,1.000000
25%,3.600000e+04,10.000000,1.000000,1.000000,1.000000
50%,3.600000e+04,183.000000,1.000000,1.000000,1.000000
75%,3.600000e+04,225.000000,1.000000,1.000000,1.000000
max,1.296000e+06,432037.000000,10.000000,80.000000,40.000000


In [17]:
pascal = accre[accre['PARTITION'] == 'pascal']
pascal_df = pd.DataFrame(pascal)

In [18]:
pascal = accre[accre['PARTITION'] == 'pascal']
pascal

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,172800.0,793.0,1,2,1,pascal,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,72000.0,18586.0,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,54000.0,15070.0,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40,302400.0,67776.0,1,1,1,pascal,0:0,COMPLETED
7,32880741,malanga,arline,1119.66,172800.0,29358.0,1,2,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336914,46753151,malanga,arline,29.85,172800.0,868.0,1,2,1,pascal,0:0,COMPLETED
336915,46753169,malanga,arline,28.11,172800.0,856.0,1,2,1,pascal,0:0,COMPLETED
336916,46753170,glasshouse,tamara,10007.76,72000.0,1756.0,1,5,4,pascal,0:0,COMPLETED
336917,46753171,malanga,arline,28.21,172800.0,869.0,1,2,1,pascal,0:0,COMPLETED


In [19]:
pascal['USEDMEM'].value_counts()

5.53        2363
5.54        1469
7.41        1151
7.42         182
8.05         159
            ... 
9023.48        1
4.58           1
17166.96       1
12166.56       1
4.84           1
Name: USEDMEM, Length: 49225, dtype: int64

In [20]:
pascal['STATE'].value_counts()

COMPLETED              77770
FAILED                   192
CANCELLED                 58
CANCELLED by 649321       17
CANCELLED by 785491        5
CANCELLED by 159541        1
CANCELLED by 0             1
CANCELLED by 46207         1
CANCELLED by 885613        1
Name: STATE, dtype: int64

In [21]:
pascal.describe()

,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,7.804600e+04,78046.000000,78046.000000,78046.000000,78046.000000
mean,1.736393e+05,10788.791713,1.011134,1.475732,1.163557
std,1.614180e+05,45990.094837,0.183548,1.558781,1.229651
min,6.000000e+01,0.000000,1.000000,1.000000,1.000000
25%,3.600000e+04,166.000000,1.000000,1.000000,1.000000
50%,8.640000e+04,544.000000,1.000000,1.000000,1.000000
75%,2.592000e+05,2170.750000,1.000000,2.000000,1.000000
max,1.036800e+06,702934.000000,15.000000,120.000000,60.000000


In [22]:
turing = accre[accre['PARTITION'] == 'turing']
turing_df = pd.DataFrame(turing)   

In [23]:
turing = accre[accre['PARTITION'] == 'turing']
turing

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
80,32881540,virginia,garrett,4039.36,7200.0,7205.0,1,6,1,turing,0:0,COMPLETED
91,32882556,virginia,garrett,55.39,21600.0,21605.0,1,6,1,turing,0:0,COMPLETED
92,32882567,virginia,garrett,281.12,21600.0,21605.0,1,6,1,turing,0:0,COMPLETED
93,32882568,virginia,garrett,57.32,21600.0,21617.0,1,6,1,turing,0:0,COMPLETED
172,32883336,virginia,erla,129.24,3600.0,3292.0,1,6,1,turing,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,432000.0,52.0,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70,432000.0,2091.0,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21,432000.0,243.0,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52,432000.0,673.0,1,1,1,turing,0:0,COMPLETED


In [24]:
turing['USEDMEM'].value_counts()

7.41       15491
5.54        2589
5.53        1319
5.41         988
7.42         865
           ...  
2239.59        1
2249.13        1
2220.82        1
2248.92        1
145.24         1
Name: USEDMEM, Length: 42116, dtype: int64

In [25]:
turing['STATE'].value_counts()

COMPLETED              74348
CANCELLED                194
FAILED                    90
CANCELLED by 505355       11
CANCELLED by 161909        4
OUT_OF_MEMORY              3
CANCELLED by 756088        1
CANCELLED by 785491        1
CANCELLED by 46207         1
Name: STATE, dtype: int64

In [26]:
turing['USEDMEM'].describe()

count     74653
unique    42116
top        7.41
freq      15491
Name: USEDMEM, dtype: object

In [27]:
turing.describe()

,REQTIME,USEDTIME,NODES,CPUS,GPUS
count,74653.000000,74653.000000,74653.000000,74653.000000,74653.000000
mean,48937.233869,7973.331427,1.003684,2.934899,1.076313
std,108571.203497,44043.472997,0.109310,2.150887,0.716668
min,60.000000,0.000000,1.000000,1.000000,1.000000
25%,1800.000000,14.000000,1.000000,2.000000,1.000000
50%,3600.000000,183.000000,1.000000,2.000000,1.000000
75%,43200.000000,945.000000,1.000000,4.000000,1.000000
max,950400.000000,863994.000000,10.000000,80.000000,40.000000


In [28]:
maxwell['USEDMEM'].describe()

count     125501
unique     36985
top         5.53
freq       24156
Name: USEDMEM, dtype: object

In [29]:
pascal['USEDMEM'].describe()

count     78046
unique    49225
top        5.53
freq       2363
Name: USEDMEM, dtype: object

In [30]:
turing['USEDMEM'].describe()

count     74653
unique    42116
top        7.41
freq      15491
Name: USEDMEM, dtype: object

In [ ]:
g = sns.FacetGrid(accre, col='PARTITION')
g.map(sns.scatterplot, 'USEDMEM', 'USEDTIME')

Linear Regression on the seperate partitions

In [ ]:
lm = smf.ols('USEDMEM ~ GPUS', data = turing).fit()
lm.summary()

In [ ]:
accre_regex = accre['USEDMEM'].str.replace(r'[a-zA-Z]','')
accre_regex

33,344 out of 125,502 entries are 5.53-5.54M memory used. Roughly 26%

In [ ]:
runtime_usage_per_gpu = turing[turing['USEDTIME' * 'GPUS']]

In [ ]:
Fig, ax = plt.subplots(figsize = (8, 6))
plt.bar(x = accre['PARTITION'], height = accre['USEDMEM'])
plt.xlabel('Continent')
plt.ylabel('Number of Countries')
plt.title('Countries per Continent')
plt.show()

In [ ]:
Fig, ax = plt.subplots(figsize=(15, 12))
accre[accre['PARTITION'] == 'maxwell'].plot(x='USEDTIME', y='USEDMEM', ax = ax)
accre[accre['PARTITION'] == 'pascal'].plot(x='USEDTIME', y='USEDMEM', ax = ax)
accre[accre['PARTITION'] == 'turing'].plot(x='USEDTIME', y='USEDMEM', ax = ax)
ax.legend(labels=['Maxwell', 'Pascal', 'Turing'])
plt.show()